In [ ]:
# This is mounted already on the RoadDamageDetection2024@gmail.com drive. This includes all our data you can see it by clicking the folder on the left.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 17.5 MB/s eta 0:00:00


In [ ]:
# Step 1: Import Necessary Libraries
import os
import yaml
from sklearn.model_selection import KFold
from ultralytics import YOLO
import ray

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Step 2: Load the base path for the data

base_path = "/content/drive/MyDrive/RoadDamageDetection/RDD2022-Combined/"

In [ ]:
all_train_images = []
all_train_labels = []

# List of all train directories within base_path
train_dirs = [
    "train",
]

# Collect all files in the train directories
for train_dir in train_dirs:
    full_train_dir = os.path.join(base_path, train_dir)

    # Debugging: Check if the directory exists
    if not os.path.exists(full_train_dir):
        print(f"Directory does not exist: {full_train_dir}")
        continue  # Skip if directory doesn't exist

    print(f"Processing directory: {full_train_dir}")

    for root, _, files in os.walk(full_train_dir):  # Recursively walk through directories
        for file in files:
            file_path = os.path.join(root, file)

            # Check for files containing .jpg or .txt in the name
            if ".jpg" in file:  # Assuming images contain .jpg
                all_train_images.append(file_path)
            elif ".txt" in file:  # Assuming labels contain .txt
                all_train_labels.append(file_path)

# Final check on collected files
print("All collected training images:", len(all_train_images))
print("All collected training labels:", len(all_train_labels))

Processing directory: /content/drive/MyDrive/RoadDamageDetection/RDD2022-Combined/train
All collected training images: 37032
All collected training labels: 37032


In [ ]:
# Step 4: Define Cross-Validation Parameters
# Set up the cross-validation parameters, including the number of folds, training epochs, image size, and an empty list to store results.

# Parameters for cross-validation
k = 2  # Number of folds
epochs = 1
img_size = 640
results = []

kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Step 5: Perform K-Fold Cross-Validation
# Perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(all_train_images)):
    print(f"Starting fold {fold + 1} of {k}...")

    # Get train and validation splits for this fold
    train_images = [all_train_images[i] for i in train_index]
    val_images = [all_train_images[i] for i in val_index]
    train_labels = [all_train_labels[i] for i in train_index]
    val_labels = [all_train_labels[i] for i in val_index]

    # Set the paths to training and validation data
    train_data_dir = os.path.dirname(train_images[0])
    val_data_dir = os.path.dirname(val_images[0])

    # Create a temporary YAML configuration file for this fold
    yaml_content = {
        'train': train_data_dir,
        'val': val_data_dir,
        'nc': 4,
        'names': ['D00', 'D10', 'D20', 'D40']
    }
    yaml_file = f"temp_data/fold_{fold}_road_damage.yaml"
    os.makedirs("temp_data", exist_ok=True)
    with open(yaml_file, "w") as f:
        yaml.dump(yaml_content, f)

    # Train YOLOv8 on this fold using the temporary YAML file
    model = YOLO("yolov9t.yaml")
    fold_results = model.train(
        data=yaml_file,
        epochs=epochs,
        imgsz=img_size
    )
    print(dir(fold_results))
    results.append(fold_results)

    # Optional: Delete the YAML file after training to clean up
    os.remove(yaml_file)
# Step 6: Calculate Average Performance Metrics
# After completing all folds, we calculate and print the average precision, recall, and mAP across the folds.

avg_precision = sum(r.metrics.precision for r in results) / k
avg_recall = sum(r.metrics.recall for r in results) / k
avg_map = sum(r.metrics.map_50 for r in results) / k  # For mAP@0.5

print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average mAP@0.5: {avg_map:.4f}")

Starting fold 1 of 2...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov9t.yaml, data=temp_data/fold_0_road_damage.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

KeyboardInterrupt: 

In [ ]:
import os
import yaml
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ultralytics import YOLO
from sklearn.model_selection import KFold

# Parameters for cross-validation
k = 2  # Number of folds
epochs = 1
img_size = 640
nc = 20
class_names = ['D00', 'D10', 'D20', 'D40', 'D43', 'D44', 'D50', 'Repair', 'D01', 'D11', 'Excavator', 'Cone', 'Drum', 'Tube', 'StyrofoamBox', 'Garbage', 'ClosedManhole', 'OpenManhole', 'Container', 'CardboardBox']

kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Training function for Ray Tune
def training_function(config, fold):
    # Get train and validation splits for this fold
    train_images = [all_train_images[i] for i in config["train_index"]]
    val_images = [all_train_images[i] for i in config["val_index"]]

    train_data_dir = os.path.dirname(train_images[0])
    val_data_dir = os.path.dirname(val_images[0])

   # Create temporary YAML configuration with unique filenames
    # Create temporary YAML configuration with unique filenames
    trial_id = config.get("trial_id", "default_trial_id")  # Unique trial ID
    yaml_file = f"temp_data/fold_{fold}_trial_{trial_id}_road_damage.yaml"
    os.makedirs("temp_data", exist_ok=True)
    yaml_content = {
        'train': train_data_dir,
        'val': val_data_dir,
        'nc': nc,
        'names': class_names
    }
    with open(yaml_file, "w") as f:
        yaml.dump(yaml_content, f)

    # Specify custom results directory in Google Drive
    results_dir = f"/content/drive/My Drive/RoadDamageDetection/yolo_results/fold_{fold}_trial_{trial_id}"
    os.makedirs(results_dir, exist_ok=True)

    # Train YOLO with the given hyperparameters
    model = YOLO("yolov9t.yaml")
    model.train(
        data=yaml_file,
        epochs=epochs,
        imgsz=img_size,
        lr0=config["lr0"],  # Initial learning rate
        lrf=config["lrf"],  # Final learning rate factor
        momentum=config["momentum"],  # Momentum
        weight_decay=config["weight_decay"]  # Weight decay
    )

    # Evaluate model performance
    metrics = model.val()
    precision = metrics.metrics["precision"]
    recall = metrics.metrics["recall"]
    map50 = metrics.metrics["map_50"]

    # Report results to Ray Tune
    tune.report(
    map_50=metrics.metrics["mAP50(B)"],
    precision=metrics.metrics["precision(B)"],
    recall=metrics.metrics["recall(B)"]
)

    # Clean up temporary YAML file
    os.remove(yaml_file)


# Define hyperparameter search space
search_space = {
    "lr0": tune.uniform(1e-5, 1e-1),  # Initial learning rate
    "lrf": tune.uniform(0.01, 1.0),  # Final learning rate factor
    "momentum": tune.uniform(0.6, 0.98),  # Momentum
    "weight_decay": tune.uniform(0.0, 0.001),  # Weight decay
}

# Scheduler for early stopping
scheduler = ASHAScheduler(
    metric="map_50",
    mode="max",
    max_t=epochs,
    grace_period=1,
    reduction_factor=2
)

# Perform k-fold cross-validation with Ray Tune
for fold, (train_index, val_index) in enumerate(kf.split(all_train_images)):
    print(f"Starting fold {fold + 1} of {k}...")

    # Run Ray Tune for this fold
    analysis = tune.run(
    tune.with_parameters(training_function, fold=fold),
    config={**search_space, "train_index": train_index, "val_index": val_index},
    num_samples=4,  # Test 2 values for each of 4 parameters (2^4 = 16 total combinations)
    scheduler=scheduler,
    resources_per_trial={"cpu": 1, "gpu": 1},  # Adjust resources
    storage_path=os.path.abspath("./ray_results")
)

    # Output best hyperparameters for this fold
    print(f"Best config for fold {fold + 1}: {analysis.best_config}")
    print(f"Best trial metrics: {analysis.best_result}")

# Aggregate Results
print("Hyperparameter tuning with k-fold cross-validation completed.")


2024-12-04 09:53:59,838	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Starting fold 1 of 2...
+--------------------------------------------------------------------------+
| Configuration for experiment     training_function_2024-12-04_09-53-59   |
+--------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                   |
| Scheduler                        AsyncHyperBandScheduler                 |
| Number of trials                 4                                       |
+--------------------------------------------------------------------------+

View detailed results here: /content/ray_results/training_function_2024-12-04_09-53-59

Trial status: 4 PENDING
Current time: 2024-12-04 09:54:00. Total running time: 0s
Logical resource usage: 1.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------+
| Trial name                      status           lr0        lrf     momentum     weight_d

100%|██████████| 5.35M/5.35M [00:00<00:00, 361MB/s]


(training_function pid=7137) AMP: checks passed ✅

Trial status: 1 RUNNING | 3 PENDING
Current time: 2024-12-04 09:54:30. Total running time: 30s
Logical resource usage: 1.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------+
| Trial name                      status           lr0        lrf     momentum     weight_decay |
+-----------------------------------------------------------------------------------------------+
| training_function_af6ba_00000   RUNNING    0.0242352   0.492033     0.852526      0.000845834 |
| training_function_af6ba_00001   PENDING    0.0772858   0.414959     0.667741      0.000640402 |
| training_function_af6ba_00002   PENDING    0.0253933   0.983273     0.615376      0.00076356  |
| training_function_af6ba_00003   PENDING    0.0903153   0.271316     0.89257       0.000230224 |
+------------------------------------------------------------------------------------------

2024-12-04 09:56:47,008	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-12-04 09:56:47,030	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/content/ray_results/training_function_2024-12-04_09-53-59' in 0.0200s.


Trial status: 1 RUNNING | 3 PENDING
Current time: 2024-12-04 09:56:47. Total running time: 2min 47s
Logical resource usage: 1.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------+
| Trial name                      status           lr0        lrf     momentum     weight_decay |
+-----------------------------------------------------------------------------------------------+
| training_function_af6ba_00000   RUNNING    0.0242352   0.492033     0.852526      0.000845834 |
| training_function_af6ba_00001   PENDING    0.0772858   0.414959     0.667741      0.000640402 |
| training_function_af6ba_00002   PENDING    0.0253933   0.983273     0.615376      0.00076356  |
| training_function_af6ba_00003   PENDING    0.0903153   0.271316     0.89257       0.000230224 |
+-----------------------------------------------------------------------------------------------+


2024-12-04 09:56:57,044	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-12-04 09:56:57,050	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 3 trial(s):
- training_function_af6ba_00001: FileNotFoundError('Could not fetch metrics for training_function_af6ba_00001: both result.json and progress.csv were not found at /content/ray_results/training_function_2024-12-04_09-53-59/training_function_af6ba_00001_1_lr0=0.0773,lrf=0.4150,momentum=0.6677,weight_decay=0.0006_2024-12-04_09-53-59')
- training_function_af6ba_00002: FileNotFoundError('Could not fetch metrics for training_function_af6ba_00002: both result.json and progress.csv were not found at /content/ray_results/training_function_2024-12-04_09-53-59/training_function_af6ba_00002_2_lr0=0.0254,lrf=0.9833,momentum=0.6154,weight_decay=0.0008_2024-12-04_09-53-59')
- training_function_af6ba_00003: FileNotFoundError('Could not

ValueError: To fetch the `best_config`, pass a `metric` and `mode` parameter to `tune.run()`. Alternatively, use the `get_best_config(metric, mode)` method to set the metric and mode explicitly.